In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
import missingno as msno
import re
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline

In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df.head(6).transpose()

,0,1,2,3,4,5
member_id,13599771,1491956,64687987,8446850,58471388,5337525
loan_amnt,8875,12500,33600,17000,14000,24000
funded_amnt,8875,12500,33600,17000,14000,24000
funded_amnt_inv,8875,12475,33600,17000,14000,24000
term,36 months,36 months,60 months,60 months,60 months,60 months
int_rate,18.25,8.9,18.55,9.71,16.99,17.27
installment,321.97,396.92,863.31,358.78,347.87,599.96
grade,D,A,E,B,D,C
sub_grade,D3,A5,E2,B1,D3,C5
emp_title,Inventory/Part time Sup,Bel Brands USA,Assistant Principal,Cook Childrens Medical Center,Cook,walmart distribution cntr


In [4]:
print (train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709903 entries, 0 to 709902
Data columns (total 64 columns):
member_id                      709903 non-null int64
loan_amnt                      709903 non-null float64
funded_amnt                    709903 non-null float64
funded_amnt_inv                709903 non-null float64
term                           709903 non-null object
int_rate                       709903 non-null float64
installment                    709903 non-null float64
grade                          709903 non-null object
sub_grade                      709903 non-null object
emp_title                      668721 non-null object
emp_length                     709903 non-null object
home_ownership                 709903 non-null object
annual_inc                     709899 non-null float64
verification_status            709903 non-null object
issue_d                        709903 non-null object
loan_status                    709903 non-null object
pymnt_plan          

In [14]:
#Combine into data:
train_df['source']= 'train'
test_df['source'] = 'test'
dataset=pd.concat([train_df,test_df],ignore_index=True)
dataset.shape

(887379, 65)

In [15]:
# XGboost is good at dealing with numbers but definitely not good when dealing with string. 
# So the solution is neither we drop them or transform them.
drop_elements = ['annual_inc_joint',# high missing rate
                 'dti_joint',# high missing rate
                 'verification_status_joint',# high missing rate
                 'issue_d',#The month which the loan was funded
                 'emp_title',
                 'desc',
                 'title',
                 'desc', 
                 'zip_code',
                 'addr_state',
                 'earliest_cr_line', #借款人最早报告的信用额度被打开的月份
                 'application_type',
                 'pymnt_plan'  # 只有三个值是Y
                ]
dataset.drop(drop_elements,axis=1,inplace=True)

# Mapping emp_length
emp_length_mapping = {'< 1 year': 1, '1 year': 2, '2 years': 3, '3 years': 4, '4 years': 5, '5 years': 6, '6 years': 7, '7 years': 8, '8 years': 9, '9 years': 10, '10 years': 11, '10+ years': 12}
dataset['emp_length'] = dataset['emp_length'].map(emp_length_mapping)
dataset['emp_length'] = dataset['emp_length'].fillna(0)

# Mapping term: 36 months->36
dataset['term']=dataset['term'].apply(lambda row:int(re.findall(r"\d+\.?\d*",row)[0]))

# Mapping Category feature
category_feature=['grade','sub_grade','home_ownership','verification_status','loan_status','purpose','initial_list_status']
dataset[category_feature]=dataset[category_feature].apply(LabelEncoder().fit_transform)
# 离散变量->one_hot
Nominal_Values = ['loan_status', 'home_ownership','verification_status','purpose','initial_list_status']
#One-Hot Coding¶
dataset = pd.get_dummies(dataset, columns=Nominal_Values)
    
#Handle datetime，将Feb-1985转换成1985-02-01
#     dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
#     dataset['issue_d'] = pd.to_datetime(dataset['issue_d'], format='%b-%Y')

dataset['collections_12_mths_ex_med'] = dataset['collections_12_mths_ex_med'].fillna(dataset.collections_12_mths_ex_med.median())
dataset['annual_inc'] = dataset['annual_inc'].fillna(dataset['annual_inc'].median())

#  There are seven variables with 3 missing values, this is such a small number that I will just replace NaN with most common values.
variables3=['total_acc','pub_rec']
for e in variables3:
    dataset[e]=dataset[e].replace(np.nan,dataset[e].mode()[0])
    
# variables1=['mths_since_last_major_derog','mths_since_last_record','mths_since_rcnt_il']
# for e in variables1:
#     dataset[e]=dataset[e].replace(np.nan,0)

#Handle datetime，将Feb-1985转换成1985-02-01
# dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
# dataset['earliest_cr_line'] = dataset['earliest_cr_line'].map(lambda x: 12*x.year + x.month)
# dataset['nowMonth']=12*pd.to_datetime('today').year+pd.to_datetime('today').month
# dataset['earliest_cr_line'] = dataset.nowMonth-dataset.earliest_cr_line
# dataset.drop(['nowMonth'],axis=1,inplace=True)

In [16]:
dataset['mths_since_rcnt_il']=dataset['mths_since_rcnt_il'].replace(np.nan,0)

In [32]:
# Check missing:
print(dataset.isnull().sum())

acc_now_delinq                 177476
all_util                       866007
annual_inc                          0
collection_recovery_fee             0
collections_12_mths_ex_med          0
dti                                 0
earliest_cr_line                    0
emp_length                          0
funded_amnt                         0
funded_amnt_inv                     0
grade                               0
il_util                        868762
inq_fi                         866007
inq_last_12m                   866007
installment                         0
int_rate                            0
loan_amnt                           0
max_bal_bc                     866007
member_id                           0
mths_since_last_major_derog    665676
mths_since_last_record         750326
mths_since_rcnt_il             866569
open_acc_6m                    866007
open_il_12m                    866007
open_il_24m                    866007
open_il_6m                     866007
open_rv_12m 

In [26]:
dataset[dataset['mths_since_last_major_derog']==0].shapedataset[dataset['mths_since_last_major_derog']==0].shape

(153, 65)

In [27]:
pd.value_counts(dataset.mths_since_last_major_derog).to_frame()

,mths_since_last_major_derog
45.0,3471
42.0,3444
46.0,3420
48.0,3405
43.0,3374
38.0,3357
44.0,3356
40.0,3348
37.0,3277
41.0,3277


In [16]:
dataset.columns

Index(['acc_now_delinq', 'all_util', 'annual_inc', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'dti', 'emp_length', 'funded_amnt',
       'funded_amnt_inv', 'grade', 'il_util', 'inq_fi', 'inq_last_12m',
       'installment', 'int_rate', 'loan_amnt', 'max_bal_bc', 'member_id',
       'mths_since_last_major_derog', 'mths_since_last_record',
       'mths_since_rcnt_il', 'open_acc_6m', 'open_il_12m', 'open_il_24m',
       'open_il_6m', 'open_rv_12m', 'open_rv_24m', 'out_prncp',
       'out_prncp_inv', 'policy_code', 'pub_rec', 'recoveries', 'revol_bal',
       'revol_util', 'source', 'sub_grade', 'term', 'tot_coll_amt',
       'tot_cur_bal', 'total_acc', 'total_bal_il', 'total_cu_tl',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee',
       'total_rec_prncp', 'total_rev_hi_lim', 'loan_status_0', 'loan_status_1',
       'loan_status_2', 'loan_status_3', 'loan_status_4', 'loan_status_5',
       'loan_status_6', 'loan_status_7', 'loan_status_8

In [45]:
train_df.head().transpose()

,0,1,2,3,4
member_id,13599771,1491956,64687987,8446850,58471388
loan_amnt,8875,12500,33600,17000,14000
funded_amnt,8875,12500,33600,17000,14000
funded_amnt_inv,8875,12475,33600,17000,14000
term,36 months,36 months,60 months,60 months,60 months
int_rate,18.25,8.9,18.55,9.71,16.99
installment,321.97,396.92,863.31,358.78,347.87
grade,D,A,E,B,D
sub_grade,D3,A5,E2,B1,D3
emp_title,Inventory/Part time Sup,Bel Brands USA,Assistant Principal,Cook Childrens Medical Center,Cook


In [17]:
train = dataset.loc[dataset['source']=='train']
test = dataset.loc[dataset['source']=='test']

In [18]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','acc_now_delinq'],axis=1,inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
print(train.shape, test.shape)

(709903, 82) (177476, 81)


In [20]:
train.to_csv('../dump/train_modified_label.csv',index=False)
test.to_csv('../dump/test_modified_label.csv',index=False)